# Introduction

The notebook is intended to perform a binary classification over the 'Response' label.

In [2]:
# Import Standard Modules
import pandas as pd

from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score, accuracy_score
from sklearn.linear_model import LogisticRegression

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

import xgboost as xgb

# Set Pandas Options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read Data

In [3]:
# Read data
data = pd.read_csv('./../data/marketing_campaign_prepared.csv', encoding='latin1', sep=';')

# Data Pre-processing

## Features & Label Definition

The 'ID' column does not bring any useful information.

In [4]:
# Define x and y
X = data.drop(['ID', 'Response'], axis=1)
y = data['Response']

## Train & Test Split

Since the label is characterized by a strong imbalancing in the class distribution, we need to address it carefully:
1. Ensure that the training and test sets have the same proportions of the two classes
2. Oversample the minor class (i.e., randomly duplicate examples)
3. Undersample the major class (i.e., randomly delete examples)
4. Use several metrics (e.g., Accuracy, Precision, Recall, AUC)

Use StratifiedShuffleSplit. This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds. The folds are made by preserving the percentage of samples for each class.

Note: like the ShuffleSplit strategy, stratified random splits do not guarantee that all folds will be different, although this is still very likely for sizeable datasets.

In [5]:
# Standard train & test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Define the a Stratified K-fold Shuffle Splitter
stratified_kfold = StratifiedShuffleSplit(n_splits=10,
                                          test_size=.3, 
                                          random_state=0)

## Feature Selection

In [6]:
# Numerical Features
numerical_features = ['Year_Birth', 
                      'Income', 
                      'Recency', 
                      'MntWines', 
                      'MntFruits', 
                      'MntMeatProducts', 
                      'MntFishProducts', 
                      'MntSweetProducts', 
                      'MntGoldProds', 
                      'NumDealsPurchases', 
                      'NumWebPurchases', 
                      'NumCatalogPurchases', 
                      'NumStorePurchases', 
                      'NumWebVisitsMonth']

# Categorical Text Features
categorical_text_features = ['Education', 
                             'Marital_Status']

# Categorical Numerical Features
categorical_numerical_features = ['Kidhome', 
                                  'Teenhome', 
                                  'AcceptedCmp1', 
                                  'AcceptedCmp2', 
                                  'AcceptedCmp3', 
                                  'AcceptedCmp4', 
                                  'AcceptedCmp5', 
                                  'Complain', 
                                  'Dt_Customer_month', 
                                  'Dt_Customer_dayofweek']

## Data Standardization

Transform the individual features to look more or less like standard normally distributed data: Gaussian with zero mean and unit variance.

Keep in mind that tree-based methods are scale-invariant, so data standardization is not required.

Standardization has to go after training-test split. That's because, standardizing the whole dataset and then split, would introduce into the training set some information about the mean and std of the test set. Remember to standardize the test set with the same scaler trained on the training set. This would be addressed by constructing a pipeline with the scaler as a step.

In [25]:
# Define the ColumnTransformer
column_transformer = ColumnTransformer([
    ('numerical_transformation', StandardScaler(), numerical_features),
    ('categorical_text_transformation', OrdinalEncoder(), categorical_text_features),
    ('categorical_numerical_transformation', 'passthrough', categorical_numerical_features)
], verbose_feature_names_out=True)

In [26]:
# Fit the ColumnTransformer
column_transformer.fit(X_train)

ColumnTransformer(transformers=[('numerical_transformation', StandardScaler(),
                                 ['Year_Birth', 'Income', 'Recency', 'MntWines',
                                  'MntFruits', 'MntMeatProducts',
                                  'MntFishProducts', 'MntSweetProducts',
                                  'MntGoldProds', 'NumDealsPurchases',
                                  'NumWebPurchases', 'NumCatalogPurchases',
                                  'NumStorePurchases', 'NumWebVisitsMonth']),
                                ('categorical_text_transformation',
                                 OrdinalEncoder(),
                                 ['Education', 'Marital_Status']),
                                ('categorical_numerical_transformation',
                                 'passthrough',
                                 ['Kidhome', 'Teenhome', 'AcceptedCmp1',
                                  'AcceptedCmp2', 'AcceptedCmp3',
                              

In [29]:
column_transformer.get_feature_names_out()

AttributeError: Transformer categorical_text_transformation (type OrdinalEncoder) does not provide get_feature_names_out.

In [27]:
# Transform the columns
X_train_transformed = column_transformer.transform(X_train)
X_test_transformed = column_transformer.transform(X_test)

In [13]:
# Check data transformation
X_train.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Year_Birth,1027.0,1969.664070,11.255456,1943.0,1961.0,1971.0,1978.0,1996.0
Income,1027.0,43185.667324,17129.090583,7500.0,30431.5,41658.0,54991.0,90247.0
Kidhome,1027.0,0.577410,0.534024,0.0,0.0,1.0,1.0,2.0
Teenhome,1027.0,0.546251,0.537623,0.0,0.0,1.0,1.0,2.0
Recency,1027.0,49.097371,28.943284,0.0,24.0,49.0,74.0,99.0
MntWines,1027.0,179.192795,255.855447,0.0,13.0,50.0,233.0,1218.0
MntFruits,1027.0,10.604674,15.889221,0.0,1.0,4.0,12.0,81.0
MntMeatProducts,1027.0,68.841285,99.394263,0.0,11.0,25.0,86.0,545.0
MntFishProducts,1027.0,15.414800,23.434145,0.0,2.0,6.0,17.0,120.0
MntSweetProducts,1027.0,10.433301,15.637655,0.0,1.0,4.0,13.0,80.0


In [28]:
# Check data transformation
X_train_transformed

array([[-0.77014129, -0.87243071,  1.0682162 , ...,  0.        ,
         6.        ,  6.        ],
       [-0.32569585,  0.01578977, -1.6971568 , ...,  0.        ,
         3.        ,  1.        ],
       [ 0.11874959,  0.3553193 ,  1.48302215, ...,  0.        ,
         8.        ,  2.        ],
       ...,
       [ 0.0298605 ,  0.6992879 ,  0.7916789 , ...,  0.        ,
         7.        ,  0.        ],
       [-0.41458494,  0.9057041 , -0.97124639, ...,  0.        ,
         4.        ,  6.        ],
       [-1.39236491,  0.39813285, -0.97124639, ...,  0.        ,
         6.        ,  0.        ]])

# Model Definition

## Logistic Regression

First benchmark model. Use standard train & test split and fit the logistic regression.

In [ ]:
# Define the model
model_lr = LogisticRegression()

# Train the model
model_lr.fit(X_train, y_train)

# Predictions
predictions_lr = model_lr.predict(X_test)

## Bayesian Optimization with HYPEROPT

In [ ]:
# Define the Hyperparamters space for Hyperopt
hyperopt_parameters_space = {
    'max_depth': hp.quniform("max_depth", 3, 40, 2),
    'gamma': hp.uniform ('gamma', 1, 15),
    'reg_alpha' : hp.quniform('reg_alpha', 40, 180, 1),
    'reg_lambda' : hp.uniform('reg_lambda', 0, 1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.5, 1),
    'min_child_weight' : hp.quniform('min_child_weight', 0, 60, 3),
    'n_estimators': hp.quniform('n_estimators', 100, 3000, 10),
    'seed': 0
}

In [ ]:
# Define the Objective Function
def objective(space, column_transformer=column_transformer, cv=stratified_kfold, X=X, y=y, scoring='roc_auc'):
    
    # Create the estimator
    clf=xgb.XGBClassifier(objective='binary:logistic',
                          eval_metric='auc',
                          n_estimators=int(space['n_estimators']), 
                          max_depth=int(space['max_depth']), 
                          gamma=space['gamma'],
                          reg_alpha=int(space['reg_alpha']),
                          min_child_weight=int(space['min_child_weight']),
                          colsample_bytree=int(space['colsample_bytree']),
                          use_label_encoder=False)
    
    # Define the Pipeline
    pipeline = Pipeline(steps=[
        ('feature_transformation', column_transformer), 
        ('classifier', clf)
    ])
    
    # Init accuracy and precision list for K-fold
    accuracy = []
    precision = []
    
    # Train the model with K-fold
    for train_index, test_index in cv.split(X, y):
    
        # Train the model
        pipeline.fit(X[train_index], y[train_index])

        # Get predicitons
        predictions = pipeline.predict(X[test_index])
        
        # Calculate accuracy
        accuracy.append(accuracy_score(y[test_index], predictions))
        
        # Calculate precision
        precision.append(precision_score(y[test_index], predictions))
                                  
        
    # Calculate the score
    cv_score = cross_val_score(pipeline, X, y, cv=cv, scoring=scoring, n_jobs=1).mean()
    accuracy_mean = accuracy.mean()
    precision_score = precision.mean()
    
    print(cv_score)
    print(accuracy_mean)
    print(precision_score)
                
    return cv_score

In [ ]:
parameters = fmin(fn=objective,
                  space=hyperopt_parameters_space,
                  algo=tpe.suggest,
                  max_evals=50,
                  trials=Trials())